In [79]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_lattice as tfl
import matplotlib.cm as cm
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.models import Model
from keras import layers, losses
from sklearn import preprocessing


from keras.models import Sequential
from keras.layers import InputLayer, LSTM, Dense, RepeatVector, TimeDistributed
from keras.utils import plot_model

In [80]:
try:
    # enable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type == 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

## Load Data

In [81]:
DATA_DIR_SLOW = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaSlow.npy"
DATA_DIR_NORMAL = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaNormal.npy"
DATA_DIR_C_NAMES = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaColumnNames.npy"

# Set style for matplotlib
plt.style.use("Solarize_Light2")

In [82]:
data_normal = np.load(DATA_DIR_NORMAL)
data_slow = np.load(DATA_DIR_SLOW)
column_names = np.load(DATA_DIR_C_NAMES)

In [83]:
print(column_names)

['action' 'machine_nameKuka Robot_apparent_power'
 'machine_nameKuka Robot_current' 'machine_nameKuka Robot_frequency'
 'machine_nameKuka Robot_phase_angle' 'machine_nameKuka Robot_power'
 'machine_nameKuka Robot_power_factor'
 'machine_nameKuka Robot_reactive_power' 'machine_nameKuka Robot_voltage'
 'sensor_id1_AccX' 'sensor_id1_AccY' 'sensor_id1_AccZ' 'sensor_id1_GyroX'
 'sensor_id1_GyroY' 'sensor_id1_GyroZ' 'sensor_id1_q1' 'sensor_id1_q2'
 'sensor_id1_q3' 'sensor_id1_q4' 'sensor_id1_temp' 'sensor_id2_AccX'
 'sensor_id2_AccY' 'sensor_id2_AccZ' 'sensor_id2_GyroX' 'sensor_id2_GyroY'
 'sensor_id2_GyroZ' 'sensor_id2_q1' 'sensor_id2_q2' 'sensor_id2_q3'
 'sensor_id2_q4' 'sensor_id2_temp' 'sensor_id3_AccX' 'sensor_id3_AccY'
 'sensor_id3_AccZ' 'sensor_id3_GyroX' 'sensor_id3_GyroY'
 'sensor_id3_GyroZ' 'sensor_id3_q1' 'sensor_id3_q2' 'sensor_id3_q3'
 'sensor_id3_q4' 'sensor_id3_temp' 'sensor_id4_AccX' 'sensor_id4_AccY'
 'sensor_id4_AccZ' 'sensor_id4_GyroX' 'sensor_id4_GyroY'
 'sensor_id4_GyroZ

## Data Setup

In [84]:
# Normalization train on train set transform both
scaler = preprocessing.StandardScaler().fit(data_normal)
data_train_scaled = scaler.transform(data_normal)
data_test_scaled = scaler.transform(data_slow[:,:-1]) # drop the last colum because it is just 1 that represents anomaly "true"

In [85]:
# take part of data to gather results faster
percentage = 0.10
data_train = data_train_scaled[0:int(percentage*data_train_scaled.shape[0]),:]
data_test = data_test_scaled[0:int(percentage*data_test_scaled.shape[0]),:]
print(f"train data shape: {data_train.shape} \n test data shape: {data_test.shape} ")

train data shape: (23379, 86) 
 test data shape: (4153, 86) 


## Model

In [86]:
class AdversarialAutoencoderGenerator(tf.keras.Model):
    def __init__(
        self,
        encoding_dims,
        input_size,
        input_channels,
        step_channels=16,
        nonlinearity=tf.keras.layers.LeakyReLU(0.2),
    ):
        super(AdversarialAutoencoderGenerator, self).__init__()
        encoder = [
            tf.keras.layers.Conv1D(step_channels, 5, strides=2, padding='same', activation=nonlinearity)
        ]
        size = input_size // 2
        channels = step_channels
        while size > 1:
            encoder.append(
                tf.keras.layers.Conv1D(channels * 4, 5, strides=4, padding='same')
            )
            encoder.append(tf.keras.layers.BatchNormalization())
            encoder.append(nonlinearity)
            channels *= 4
            size = size // 4
        self.encoder = tf.keras.Sequential(encoder)
        self.encoder_fc = tf.keras.layers.Dense(encoding_dims)
        self.decoder_fc = tf.keras.layers.Dense(step_channels * size)
        decoder = []
        size = 1
        channels = step_channels
        while size < input_size // 2:
            decoder.append(
                tf.keras.layers.Conv1DTranspose(channels * 4, 5, strides=4, padding='same')
            )
            decoder.append(tf.keras.layers.BatchNormalization())
            decoder.append(nonlinearity)
            channels *= 4
            size *= 4
        decoder.append(tf.keras.layers.Conv1DTranspose(input_channels, 5, strides=2, padding='same'))
        self.decoder = tf.keras.Sequential(decoder)

    def sample(self, noise):
        noise = self.decoder_fc(noise)
        noise = tf.reshape(noise, (-1, 1, noise.shape[1]))
        return self.decoder(noise)

    def call(self, x):
        if self.training:
            encoding = self.encoder(x)
            encoding = tf.reshape(encoding, (-1, tf.reduce_prod(encoding.shape[1:])))
            encoding = self.encoder_fc(encoding)
            return self.sample(encoding), encoding
        else:
            return self.sample(x)

In [87]:
class AdversarialAutoencoderDiscriminator(tf.keras.Model):
    def __init__(self, input_dims, non_linearity=tf.keras.layers.LeakyReLU(0.2)):
        super(AdversarialAutoencoderDiscriminator, self).__init__()
        model = [tf.keras.layers.Dense(input_dims // 2, activation=non_linearity)]
        size = input_dims // 2
        while size > 16:
            model.append(tf.keras.layers.Dense(size // 2))
            model.append(tf.keras.layers.BatchNormalization())
            model.append(non_linearity)
            size = size // 2
        model.append(tf.keras.layers.Dense(1))
        self.model = tf.keras.Sequential(model)

    def call(self, x):
        return self.model(x)

## Loss functions

In [88]:
class AdversarialAutoencoderGeneratorLoss(tf.keras.losses.Loss):
    def __init__(self):
        super(AdversarialAutoencoderGeneratorLoss, self).__init__()

    def call(self, real_inputs, gen_inputs, dgz):
        mse_loss = tf.keras.losses.MeanSquaredError()(gen_inputs, real_inputs)
        bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(dgz, tf.ones_like(dgz))
        loss = 0.999 * mse_loss + 0.001 * bce_loss
        return loss

    def train_ops(
        self,
        generator,
        discriminator,
        optimizer_generator,
        real_inputs,
        device,
        batch_size,
        labels=None,
    ):
        with tf.GradientTape() as tape:
            recon, encodings = generator(real_inputs)
            dgz = discriminator(encodings)
            loss = self.call(real_inputs, recon, dgz)
        gradients = tape.gradient(loss, generator.trainable_variables)
        optimizer_generator.apply_gradients(zip(gradients, generator.trainable_variables))
        return loss.numpy()

In [89]:
class AdversarialAutoencoderDiscriminatorLoss(tf.keras.losses.Loss):
    def __init__(self):
        super(AdversarialAutoencoderDiscriminatorLoss, self).__init__()

    def call(self, dx, dgz):
        bce_loss_real = tf.keras.losses.BinaryCrossentropy(from_logits=True)(dx, tf.ones_like(dx))
        bce_loss_fake = tf.keras.losses.BinaryCrossentropy(from_logits=True)(dgz, tf.zeros_like(dgz))
        loss = 0.5 * bce_loss_real + 0.5 * bce_loss_fake
        return loss

    def train_ops(
        self,
        generator,
        discriminator,
        optimizer_discriminator,
        real_inputs,
        device,
        batch_size,
        labels=None,
    ):
        _, encodings = generator(real_inputs)
        noise = tf.random.normal((real_inputs.shape[0], generator.encoding_dims), device=device)
        with tf.GradientTape() as tape:
            dx = discriminator(noise)
            dgz = discriminator(encodings)
            loss = self.call(dx, dgz)
        gradients = tape.gradient(loss, discriminator.trainable_variables)
        optimizer_discriminator.apply_gradients(zip(gradients, discriminator.trainable_variables))
        return loss.numpy()

## Trainer

In [90]:
losses = [
    AdversarialAutoencoderGeneratorLoss(),
    AdversarialAutoencoderDiscriminatorLoss(),
]

In [91]:
network = {
    "generator": {
        "name": AdversarialAutoencoderGenerator,
        "args": {"encoding_dims": 128, "input_size": 86, "input_channels": 1},
        "optimizer": {"name": tf.keras.optimizers.Adam, "args": {"learning_rate": 0.0002, "beta_1": 0.5, "beta_2": 0.999}},
    },
    "discriminator": {
        "name": AdversarialAutoencoderDiscriminator,
        "args": {"input_dims": 128,},
        "optimizer": {"name": tf.keras.optimizers.Adam, "args": {"learning_rate": 0.0002, "beta_1": 0.5, "beta_2": 0.999}},
    },
}

In [92]:

class Trainer:
    def __init__(self, network, losses, sample_size, epochs, device):
        self.generator = network["generator"]["name"](**network["generator"]["args"])
        self.discriminator = network["discriminator"]["name"](**network["discriminator"]["args"])
        self.generator_optimizer = network["generator"]["optimizer"]["name"](**network["generator"]["optimizer"]["args"])
        self.discriminator_optimizer = network["discriminator"]["optimizer"]["name"](**network["discriminator"]["optimizer"]["args"])
        self.losses = losses
        self.sample_size = sample_size
        self.epochs = epochs
        self.device = device

    def train(self, train_data):
        for epoch in range(self.epochs):
            total_loss = 0.0
            num_batches = 0

            for batch in train_data:
                real_inputs = batch  # Assuming each batch is a tensor of shape (batch_size, input_dims)

                with tf.device(self.device):
                    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                        recon, encodings = self.generator(real_inputs)
                        dx = self.discriminator(real_inputs)
                        dgz = self.discriminator(encodings)

                        generator_loss = self.losses.generator_loss(real_inputs, recon, dgz)
                        discriminator_loss = self.losses.discriminator_loss(dx, dgz)

                    generator_gradients = gen_tape.gradient(generator_loss, self.generator.trainable_variables)
                    discriminator_gradients = disc_tape.gradient(discriminator_loss, self.discriminator.trainable_variables)

                    self.generator_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
                    self.discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))

                    total_loss += generator_loss + discriminator_loss
                    num_batches += 1

            average_loss = total_loss / num_batches
            print(f"Epoch {epoch+1}/{self.epochs}, Loss: {average_loss:.4f}")

In [93]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [98]:
trainer = Trainer(network, losses, sample_size=64, epochs=128, device="/GPU")

In [101]:
trainer.train(data_train)

AttributeError: Exception encountered when calling layer "adversarial_autoencoder_generator_5" (type AdversarialAutoencoderGenerator).

'AdversarialAutoencoderGenerator' object has no attribute 'training'

Call arguments received by layer "adversarial_autoencoder_generator_5" (type AdversarialAutoencoderGenerator):
  • x=tf.Tensor(shape=(86,), dtype=float32)